<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Ref_Map/KZ_BuildRef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy --quiet
!pip install pybiomart --quiet
!pip install python-igraph --quiet
!pip install louvain --quiet
!pip install pynndescent --quiet


In [ ]:
import h5py
import numpy as np
import scipy as sci
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd
import pybiomart
from tqdm import tqdm
import urllib.request
from IPython.display import clear_output
from matplotlib.pyplot import rc_context
from scipy import stats as st
import logging
logging.Logger('my_log').setLevel('INFO')

############# SPECIFIC CONFIG #############
working_dir = r"C:\Users\dennis.tabuena\Gladstone Dropbox\Dennis Tabuena\0_Projects\_SeqRef"
os.chdir(working_dir)
zalocusky_url = 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE167497&format=file'
geo_zalo_filename = 'zalocusky_indiv.tar'

import urllib
response = urllib.request.urlretrieve('https://raw.githubusercontent.com/dtabuena/Resources/main/Matplotlib_Config/Load_FS6.py','Load_FS6.py')
%run Load_FS6.py



In [ ]:
def trim_key(k):
    floxed_dict = {'GSM5106175_YH_KZ03_01':('E3fKI_Syn_Cre602_15m','GSM5106175_602_E3fKI_15_XX'),
                   'GSM5106176_YH_KZ03_03':('E4fKI_Syn_Cre475_15m','GSM5106176_475_E4fKI_15_XX')}
    for f in floxed_dict.keys():
        if f in k: return floxed_dict[f][1]
    k = k.replace('_raw_gene_bc_matrices_h5.h5',"")
    return k

def query_capitilaziation(gene,adata):
    try:
        return adata.var.index[ [g.lower() for g in list(adata.var.index)].index(gene.lower()) ]
    except:
        return gene + ' not_found'

def z_score(x,axis=-1):
    x=np.array(x)
    return (x-np.mean(x,axis=axis))/np.std(x,axis=axis)



In [ ]:
def pull_gene_annots(csv_loc='./mmusculus_coding_noncoding.csv',
                     my_git='https://raw.githubusercontent.com/dtabuena/Resources/main/Genetics/mmusculus_coding_noncoding.csv',
                     biomart_name='mmusculus',
                     biomart_keys=["ensembl_gene_id", "chromosome_name","transcript_biotype","external_gene_name","peptide"]):

    if os.path.exists('./mmusculus_coding_noncoding.csv'):
        print( 'Use local copy of musmus')
        annot_dd = pd.read_csv('./mmusculus_coding_noncoding.csv').set_index("external_gene_name")
    else:
        try:
            print( 'attempting to pull mus mus from git...')
            musmus_link = 'https://raw.githubusercontent.com/dtabuena/Resources/main/Genetics/mmusculus_coding_noncoding.csv'
            filename = './mmusculus_coding_noncoding.csv'
            urllib.request.urlretrieve(musmus_link, filename)
            annot_dd = pd.read_csv('./mmusculus_coding_noncoding.csv').set_index("external_gene_name")
        except:
            print('attempting to pull mus mus from biomart...')
            annot = sc.queries.biomart_annotations("mmusculus",["ensembl_gene_id", "chromosome_name","transcript_biotype","external_gene_name"],).set_index('ensembl_gene_id')
            uniq_inds = list(set(list(annot.index)))
            for r in tqdm(uniq_inds):
                match_bool = annot.index.str.contains(r)
                if np.sum(match_bool)>1:
                    new_val ='__'.join(list(annot.loc[r,'transcript_biotype']))
                    annot.at[r,'transcript_biotype']=new_val
            annot['is_coding']= annot.transcript_biotype.str.contains('coding')
            annot_dd = annot.drop_duplicates().set_index("external_gene_name")
            annot_dd.to_csv('./mmusculus_coding_noncoding.csv')

    coding_list = annot_dd.index[ annot_dd['is_coding'] ].to_list()
    return coding_list, annot_dd


def preprocess_andata10x(adata_og,pct_mito=0.25,min_genes=500,max_genes=2400,min_counts=500,max_counts=4500,keep_NC=False):

    print('pulling gene annotations...')
    coding_list, _ = pull_gene_annots()
    adata_og.var['mt'] = adata_og.var_names.str.startswith('mt-')
    adata_og.var['coding'] = [gene in coding_list for gene in adata_og.var_names]
    sc.pp.calculate_qc_metrics(adata_og, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

    adata_QC = adata_og.copy()

    print('Filtering...')
    adata_QC = adata_QC[adata_QC.obs.pct_counts_mt < pct_mito, :]
    print(str(np.sum(adata_og.obs.pct_counts_mt <pct_mito)) + f' cells with >{pct_mito}% mt removed')

    if keep_NC:
        print('keep noncoding')
    else:
        adata_QC = adata_QC[:, adata_QC.var.coding]
        print(str(np.sum(np.logical_not(adata_og.var.coding))) + ' non coding genes removed')

    sc.pp.filter_cells(adata_QC, min_genes=min_genes)
    sc.pp.filter_cells(adata_QC, max_genes=max_genes)
    sc.pp.filter_cells(adata_QC, min_counts=min_counts)
    sc.pp.filter_cells(adata_QC, max_counts=max_counts)
    fig,ax=plt.subplots(1,figsize=(1.5,1.5))
    sc.pl.scatter(adata_QC, x='total_counts', y='n_genes_by_counts',ax=ax)

    return adata_QC,adata_og

def high_var_genes_dim_reduc(adata,min_mean = 0.25,max_mean = 4,min_disp=0.55):
    ''' The gene expression matrices were then log-normalized with a scale factor of 10,000,
    using the Seurat NormalizeData function57,58. Highly dispersed genes were selected using
    the Seurat FindVariableGenes function57,58,filtering for an average expression range of
    0.25–4 and a minimum dispersion of 0.55, resulting in a list of 2,197 genes.'''
    adata.raw = adata
    sc.pp.normalize_total(adata, target_sum=10000)
    sc.pp.log1p(adata)
    adata.uns['log1p'] = {'base': None}
    print(adata.uns['log1p'])
    sc.pp.highly_variable_genes(adata, min_mean=min_mean, max_mean=max_mean, min_disp=min_disp)
    with rc_context({'figure.figsize': (1.5, 1.5)}):
        sc.pl.highly_variable_genes(adata)
    plt.tight_layout()
    print(np.sum(adata.var['highly_variable']),'hv genes')


    #### PCA
    sc.tl.pca(adata, svd_solver='arpack',n_comps=50)
    fig,ax=plt.subplots(figsize=(1,1))
    ax.plot(adata.uns['pca']['variance_ratio'][:25],'ok',markersize=1)
    quiet_PCA_plots(adata,['E_type','age_bin','mouse_ID'],pc_pairs=[(0,1),(2,3)])

    return adata

def quiet_PCA_plots(adata,key_list,figsize=(2,2),pc_pairs=[(0,1)]):
    fig,ax=plt.subplots(1*len(pc_pairs),len(key_list),figsize=(figsize[0]*len(key_list),figsize[1]*len(pc_pairs)))
    for ip,pair in enumerate(pc_pairs):
        if len(key_list) == 1: ax=[ax]
        for key_ind,key in enumerate(key_list):
            key_types = sorted(list(set( adata.obs[key] )))
            for k in key_types:
                is_k = adata.obs[key]==k
                ax[ip,key_ind].scatter(adata.obsm['X_pca'][is_k,pair[0]],adata.obsm['X_pca'][is_k,pair[1]],s=2,marker='.',linewidth=0,edgecolors=None,label=k)
                ax[ip,key_ind].set_xlabel(f'PC{pair[0]}')
                ax[ip,key_ind].set_ylabel(f'PC{pair[1]}')
            if len(key_types)<8: ax[ip,key_ind].legend(key_types,loc='best',markerscale=3)
            ax[ip,key_ind].set_title(key)
            plt.tight_layout()
    return None


def umap_and_cluster(adata, n_neighbors=10, n_pcs=20,resolution=.6,plot_keys=['Cluster (nn)'],size = 1,to_plot=True):
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs,random_state=42, use_rep='X_pca')
    sc.tl.louvain(adata,resolution=resolution,random_state=42)
    sc.tl.paga(adata)
    sc.tl.umap(adata,random_state=42)
    adata.obs['Cluster (nn)']= adata.obs['louvain']
    if to_plot:
        with rc_context({'figure.figsize': (2.5, 2.5)}):
            sc.pl.umap(adata,add_outline=False, legend_loc='on data', color=plot_keys,size=size)
    return adata

def explore_umap(adata_GABA,key_list=[],size=1,legend_loc=None):
    with rc_context({'figure.figsize': (1.5,1.5)}):
        sc.pl.umap(adata_GABA, legend_loc=legend_loc, color=key_list,vmin=-0.5,size=size,cmap='Purples') # add_outline=True,
        plt.tight_layout()




def marker_analysis(adata):
    #### Get Marker Genes
    sc.tl.rank_genes_groups(adata, 'louvain', method='wilcoxon',key_added='maker_genes',pts=True,use_raw =True)
    sc.tl.rank_genes_groups(adata, 'louvain', method='logreg',pts=True,use_raw =True)
    maker_genes_df = pd.DataFrame(adata.uns['maker_genes']['names'])
    maker_genes_df_LFC = pd.DataFrame(adata.uns['maker_genes']['logfoldchanges'])
    top_30 = maker_genes_df[:30]
    lfc_thresh=4
    marker_genes = [m for m in set(top_30[maker_genes_df_LFC[:30]>lfc_thresh].values.flatten()) if isinstance(m,str)]
    sc.pl.stacked_violin(adata, marker_genes, groupby='louvain');
    return marker_genes






In [ ]:

def show_clusters(adata_GABA, key, cmap='tab20', legend_fontsize='small',size = 0.4):
    assert key in adata_GABA.obs, f"Key '{key}' not found in adata_GABA.obs."

    umap_coords = adata_GABA.obsm['X_umap']


    # Create the figure with subplots for scatter and legend
    fig, (ax_scatter, ax_legend) = plt.subplots(1, 2, figsize=(3.5, 2.5), gridspec_kw={'width_ratios': [5, 1]})

    # Get cluster colors
    colors = adata_GABA.obs[key].values.astype(float)
    scatter = ax_scatter.scatter(
        umap_coords[:, 0], umap_coords[:, 1],
        c=colors, cmap=cmap, s=size, vmin=-0.5, edgecolor='none'  # Set edgecolor to 'none'
    )

    # Set equal aspect ratio to make axes square
    ax_scatter.set_aspect('equal', adjustable='box')
    ax_scatter.set_xlabel('umap 1',ha='left')
    ax_scatter.xaxis.set_label_coords(0, 0.0)
    ax_scatter.set_ylabel('umap 2',ha='left')
    ax_scatter.yaxis.set_label_coords(0.0, 0)
    ax_scatter.set_xticks([])
    ax_scatter.set_yticks([])

    unique_clusters = np.unique(colors)
    handles = []
    labels = []

    # Create custom legend handles for unique clusters
    for cluster in unique_clusters:
        color_i = scatter.cmap(scatter.norm(cluster))
        handle = plt.Line2D([], [], marker='o', linestyle='None', color=color_i,
                             markerfacecolor=color_i,
                             markersize=4.5)  # Smaller markers
        handles.append(handle)
        labels.append(f'Cluster {int(cluster)}')

    # Turn off the axis for the legend subplot
    ax_legend.axis('off')

    # Add the custom legend to the legend axis with specified font size
    ax_legend.legend(handles, labels, loc='center', fontsize=legend_fontsize, markerscale=0.75,
                     ncol=1, frameon=False)
    plt.tight_layout()
    plt.show()

    return fig, (ax_scatter, ax_legend)

# Example calls to the function
# show_clusters(adata_GABA, 'Cluster (nn)')


def show_gene_loading(adata, gene, cmap='Purples',size = 0.4,alpha=0.8):
    assert gene in adata.var_names, f"Gene '{gene}' not found in adata.var_names."

    umap_coords = adata.obsm['X_umap']


    # Get gene expression values for the specified gene
    gene_expression = adata[:, gene].X.toarray().flatten()  # Convert to dense array and flatten

    # Create the figure
    fig = plt.figure(figsize=(3, 1.5))

    # Define positions for scatter and colorbar axes
    ax_scatter = fig.add_axes([0.0, 0.1, 0.5, 0.8])  # [left, bottom, width, height]
    ax_cbar = fig.add_axes([0.50, 0.1, 0.01, 0.8])  # Slightly to the right of the scatter plot

    # Scatter plot
    scatter = ax_scatter.scatter(
        umap_coords[:, 0], umap_coords[:, 1],
        c=gene_expression, cmap=cmap, s=size, edgecolor='none',alpha=alpha,vmin=-.45,vmax=2.75)

    # Set equal aspect ratio to make axes square
    ax_scatter.set_aspect('equal', adjustable='box')
    ax_scatter.set_xlabel('umap 1',ha='left')
    ax_scatter.xaxis.set_label_coords(0, 0.0)
    ax_scatter.set_ylabel('umap 2',ha='left')
    ax_scatter.yaxis.set_label_coords(0.0, 0)
    ax_scatter.set_xticks([])
    ax_scatter.set_yticks([])

    # Colorbar
    cbar = fig.colorbar(scatter, cax=ax_cbar, orientation='vertical')
    cbar.set_label(f'stdev/mean', rotation=270, labelpad=10)
    ax_scatter.set_title(gene.upper())
    # Ensure colorbar is labeled
    # ax_cbar.set_ylabel(f'{gene} Expression', rotation=270, labelpad=10)

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()

    return fig, (ax_scatter, ax_cbar)

# Example call to the function
# show_gene_loading(adata_GABA, 'Sst')

# cluster_fig, ax = show_clusters(adata_GABA, 'Cluster (nn)')
# cluster_fig.savefig("cluster_plot.svg", format="svg")
# cluster_fig.savefig("cluster_plot.jpeg", format="jpeg")
# show_gene_loading(adata_full, 'Prox1')

In [ ]:

################# INITIALIZE DIRECTORY DOWNLOAD FROM GEO
os.chdir(working_dir)
os.makedirs('./indiv_animal_results', exist_ok=True)
urllib.request.urlretrieve(zalocusky_url, './indiv_animal_results/'+geo_zalo_filename)
my_tar = tarfile.open('./indiv_animal_results/'+geo_zalo_filename)
my_tar.extractall('./indiv_animal_results') # specify which folder to extract to
my_tar.close()
# for f in os.listdir('./indiv_animal_results'):
#     print(f)


In [ ]:

################# Read, Combine, and Sample Split multiple 10x's

adata_dict = {}
for f in os.listdir('./indiv_animal_results'):
    if '.h5' in f:
        a = sc.read_10x_h5('./indiv_animal_results/'+f)
        a.var_names_make_unique()
        sample_code = trim_key(f)
        a.obs['age_bin'] = str(int(np.ceil( int(sample_code.split("_")[3])/5)*5))+'m'
        a.obs['E_type'] = sample_code.split("_")[2]
        a.obs['mouse_ID'] = sample_code.split("_")[1]
        a.obs['well'] = sample_code.split("_")[4]
        a.obs['GSM'] = sample_code.split("_")[0]
        adata_dict[sample_code.split("_")[0]] = a
adata = ad.concat(adata_dict,axis = 0,label="Sample",index_unique="_")
adata.obs.E_type
# adata = adata[['fKI' not in t for t in adata.obs.E_type], :]
adata_dict = {}
clear_output()
adata.write_h5ad(filename='./kz_adata_raw.h5')


adata_QC = preprocess_andata10x(adata,keep_NC=True)[0]
adata_QC.write_h5ad(filename='./kz_adata_qc.h5')




In [ ]:
############## QC Filter #####################
adata_full = adata_QC.copy()
age_dict = {'5m':'05m', '10m': '10m','15m': '15m','20m': '20m'}
adata_full.obs['age_bin'] = [ age_dict[a] for a in adata_full.obs['age_bin'] ]

sc.pp.filter_genes(adata_full, min_counts=50)

############## Dim Reduction #####################
adata_full = high_var_genes_dim_reduc(adata_full)
adata_full.write_h5ad(filename='./adata_full.h5')
print(adata_full.uns['log1p'])



In [ ]:
############## Dim Reduction #####################
adata_full = ad.read_h5ad('./adata_full.h5')
adata_full = umap_and_cluster(adata_full, n_neighbors=15, n_pcs=20,resolution=.6,to_plot=False)
show_clusters(adata_full, 'Cluster (nn)')

In [ ]:
tab40_colors[0]

In [ ]:
tab20b_colors = plt.cm.get_cmap('tab20b').colors
tab20c_colors = plt.cm.get_cmap('tab20c').colors
tab40_colors = np.concatenate([tab20c_colors, tab20b_colors[4:]])
tab40 = mpl.colors.ListedColormap(tab40_colors)



show_clusters(adata_full, 'Cluster (nn)',cmap=tab40)
# show_gene_loading(adata_full, 'Sst')
# show_gene_loading(adata_full, 'Pvalb')
# show_gene_loading(adata_full, 'Vip')
# show_gene_loading(adata_full, 'Reln')
# show_gene_loading(adata_full, 'Apoe')
show_gene_loading(adata_full, 'Syn1',size=.1,alpha=1)
show_gene_loading(adata_full, 'Slc17a7',size=.1,alpha=1)
show_gene_loading(adata_full, 'Prox1',size=.1,alpha=1)
show_gene_loading(adata_full, 'Pdzd2',size=.1,alpha=1)


# show_gene_loading(adata_full, 'Gfap')
# show_gene_loading(adata_full, 'Gad1')
# show_gene_loading(adata_full, 'Gad2')




In [ ]:
DGC_adata = adata_full[adata_full.obs['louvain'].isin(['0', '2', '4']), :]
DGC_adata.obsm = {}
DGC_adata.uns = {}
sc.tl.pca(DGC_adata, n_comps=50)
sc.pp.scale(DGC_adata)
DGC_adata = umap_and_cluster(DGC_adata, n_neighbors=25, n_pcs=30,resolution=.6,to_plot=False)
show_clusters(DGC_adata, 'Cluster (nn)')

In [ ]:
sc.pl.pca_variance_ratio(DGC_adata)
DGC_adata.write_h5ad(filename='./DGC_adata.h5')

In [ ]:
# show_clusters(DGC_adata, 'Cluster (nn)')
show_gene_loading(DGC_adata, 'Apoe',size = 0.5,alpha=0.5)
show_gene_loading(DGC_adata, 'Syn1',size = 0.5,alpha=0.5)
show_gene_loading(DGC_adata, 'Slc17a7',size = 0.5,alpha=0.5)
show_gene_loading(DGC_adata, 'Prox1',size = 0.5,alpha=0.5)
show_gene_loading(DGC_adata, 'Pdzd2',size = 0.5,alpha=0.5)


In [ ]:
sc.tl.rank_genes_groups(DGC_adata, groupby='louvain', method='wilcoxon')

In [ ]:
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

def calculate_enrichment_with_stats(adata, group_var, cluster_var, alpha=0.05):
    # Contingency table of counts
    contingency_table = pd.crosstab(adata.obs[cluster_var], adata.obs[group_var])
    total_cells = contingency_table.values.sum()

    # Observed and expected proportions
    cluster_totals = contingency_table.sum(axis=1).values[:, None]  # Total in each cluster
    group_totals = contingency_table.sum(axis=0).values[None, :]  # Total in each group
    expected = cluster_totals * group_totals / total_cells  # Expected counts
    enrichment = contingency_table / expected  # Observed-to-Expected ratio

    # Perform Fisher's Exact Test for each cell
    p_values = []
    for cluster_idx in range(contingency_table.shape[0]):
        for group_idx in range(contingency_table.shape[1]):
            obs = contingency_table.iloc[cluster_idx, group_idx]
            rest_cluster = cluster_totals[cluster_idx, 0] - obs
            rest_group = group_totals[0, group_idx] - obs
            rest_total = total_cells - obs - rest_cluster - rest_group

            # Construct the 2x2 table for Fisher's test
            table = np.array([[obs, rest_group], [rest_cluster, rest_total]])
            _, p_value = fisher_exact(table, alternative="two-sided")
            p_values.append(p_value)

    # FDR correction for multiple testing
    p_values_corrected = multipletests(p_values, method="fdr_bh")[1]

    # Format results
    p_corrected_matrix = pd.DataFrame(
        np.array(p_values_corrected).reshape(contingency_table.shape),
        index=contingency_table.index,
        columns=contingency_table.columns,
    )

    return enrichment, p_corrected_matrix



In [ ]:
def p_to_astk(p_val):
        astk_range = [0.05,0.01,0.001,0.0001]
        num_astk = ["*" for t in astk_range if p_val <= t ]
        astk = "".join(num_astk)
        if len(astk)==0:
            astk='n.s.'
        return astk


In [ ]:
def plot_helicopter(df, pvalues, title, cbar_label="Enrichment", ax=None):
    """
    Plots a helicopter plot (heatmap) with enrichment values and p-values annotated.

    Args:
        df (pd.DataFrame): DataFrame containing enrichment values.
        pvalues (pd.DataFrame): DataFrame containing corresponding p-values.
        title (str): Title for the plot.
        cbar_label (str): Label for the color bar.
        ax (matplotlib.axes.Axes, optional): Axes to plot on. Creates a new figure if None.
    """
    # Create a new figure only if ax is None
    if ax is None:
        fig, ax = plt.subplots(figsize=(2, 1.5))

    # Calculate color scale limits for symmetry about 1
    vrange = abs(df.values - 1).max()
    vmin = 1 - vrange
    vmax = 1 + vrange

    # Function to format p-values
    def format_pvalue(p):
        if p < 0.001:
            return f"\n(p={p:.1e})"  # Scientific notation for p < 0.001
        else:
            return f"\n(p={p:.3f})"  # Decimal format for p >= 0.001

    # Initialize an empty DataFrame to hold the annotations
    annotations = pd.DataFrame(index=df.index, columns=df.columns, dtype=object)

    # Format the enrichment values and p-values
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            enrichment = f"{df.iloc[i, j]:.2f}"
            pvalue = p_to_astk(pvalues.iloc[i, j])  # Use p_to_astk for p-value abbreviation
            annotations.iloc[i, j] = enrichment + "\n" + pvalue

    sns.heatmap(
        df,
        annot=annotations.values,
        fmt="",
        cmap="coolwarm",
        cbar_kws={"label": cbar_label},
        vmin=vmin,
        vmax=vmax,
        ax=ax,  # Use the provided axes or create a new one
    )
    ax.set_title(title)
    ax.set_xlabel(None)
    ax.set_ylabel("Clusters")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

    # Adjust layout only if no external ax is provided
    if ax is None:
        plt.tight_layout()
        plt.show()

    return ax



# age_bin_enrichment, age_bin_corrected_p_values = calculate_enrichment_with_stats(DGC_adata, group_var="age_bin", cluster_var="louvain")
# e_type_enrichment, e_type_corrected_p_values = calculate_enrichment_with_stats(DGC_adata, group_var="E_type", cluster_var="louvain")


# # Generate helicopter plots
# fig,ax = plt.subplots(1,2,figsize=(8,2))
# plot_helicopter(age_bin_enrichment, age_bin_corrected_p_values, title="Helicopter Plot: Age Bin Enrichment Across Clusters",ax=ax[0])
# plot_helicopter(e_type_enrichment, e_type_corrected_p_values, title="Helicopter Plot: E_type Enrichment Across Clusters",ax=ax[1])
# plt.tight_layout()

In [ ]:
def plot_enrichment_lines(enrichment, ax=None):
    enrichment=enrichment.T
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))  # Create a new figure if no ax is provided

    # Plotting the raw enrichment values as lines for each cluster
    for cluster in enrichment.columns:
        ax.plot(enrichment.index, enrichment[cluster], label=cluster,marker='o',linewidth=1,markersize=3)

    # Adding labels and title
    # ax.set_xlabel("E_type and Age_bin")
    ax.set_ylabel("Raw Enrichment")
    # ax.set_title("Raw Enrichment Across Clusters")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    # Add legend outside the plot
    ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), title="Clusters")

    # Adjust layout to avoid overlap
    plt.tight_layout()

    return ax


In [ ]:
DGC_adata
enrichment, corrected_p_values = calculate_enrichment_with_stats(DGC_adata, group_var="E_type_age_bin", cluster_var="louvain")


"""
E3 vs E4 Enrichment over time
"""
fig,ax = plt.subplots(1,2,figsize=(4,1.75), width_ratios=(1,1))

subset = [c for c in enrichment.columns if 'fKI' not in c and 'E3' in c]
plot_helicopter(enrichment[subset], corrected_p_values[subset], title="" ,ax=ax[0])

subset = [c for c in enrichment.columns if 'fKI' not in c and 'E4' in c]
plot_helicopter(enrichment[subset], corrected_p_values[subset], title="" ,ax=ax[1])

plt.tight_layout()

"""
SynCre Compsre
"""
fig,ax = plt.subplots(1,2,figsize=(3,1.75), width_ratios=(1,1))

subset = [c for c in enrichment.columns if '15m' in c and 'E3' in c]
plot_helicopter(enrichment[subset], corrected_p_values[subset], title="" ,ax=ax[0])

subset = [c for c in enrichment.columns if '15m' in c and 'E4' in c]
plot_helicopter(enrichment[subset], corrected_p_values[subset], title="" ,ax=ax[1])

plt.tight_layout()


In [ ]:

# enrichment_E3
enrichment_E3 = enrichment[[c for c in enrichment.columns if 'fKI' not in c and 'E3' in c]]
enrichment_E4 = enrichment[[c for c in enrichment.columns if 'fKI' not in c and 'E4' in c]]
clusters = enrichment_s.index.to_list()
num_clusters = len(clusters)

fig,ax = plt.subplots(1,num_clusters, figsize=(1*num_clusters,1.2))
for i,c in enumerate(clusters):
    x = enrichment_s.columns
    x = [a[:3] for a in x ]
    ax[i].plot(x,enrichment_E3.loc[c],label='E3',color='b',marker='o')
    ax[i].plot(x,enrichment_E4.loc[c],label='E4',color='r',marker='o')
    ax[i].set_title(f'Cluster {c}')

for a in ax:
    a.set_ylim([.5,2])
    a.axhline(1,linestyle='dotted',color='k')
ax[0].legend()
plt.tight_layout()


In [ ]:
sc.tl.dendrogram(DGC_adata, groupby='louvain')
ordered_clusters = DGC_adata.uns['dendrogram_louvain']['categories_ordered']

# Extract top-ranked genes for each cluster
top_n = 5
ranked_genes = pd.DataFrame({
    cluster: DGC_adata.uns['rank_genes_groups']['names'][cluster][:top_n]
    for cluster in ordered_clusters
})
# display(ranked_genes.head())
top_genes_all_clusters = ranked_genes.values.flatten().tolist()
top_genes_all_clusters = list(set(top_genes_all_clusters))
top_genes_all_clusters.append('Apoe')



sc.pl.rank_genes_groups_dotplot(DGC_adata, n_genes=top_n)


sc.pl.heatmap(DGC_adata, var_names=top_genes_all_clusters, groupby="louvain", cmap="viridis", dendrogram=False,figsize=(6, 1.5))


sc.pl.stacked_violin(
    DGC_adata,
    var_names=top_genes_all_clusters,
    groupby="louvain",
    swap_axes=False,
    dendrogram=False,
    figsize=(6,  1.5),
)

sc.pl.matrixplot(
    DGC_adata,
    var_names=top_genes_all_clusters,
    groupby="louvain",
    dendrogram=False,
    cmap="Blues",
    standard_scale="var",
    colorbar_title="column scaled\nexpression",
    figsize=(6,  1.5),
)

print()

In [ ]:
fig,ax = plt.subplots(figsize=(2,1))
sc.pl.violin(
    DGC_adata,
    keys='Apoe',
    groupby="louvain",
    ax=ax
)

In [ ]:
corr_method


In [ ]:
os.getcwd()

In [ ]:
marker_genes = DGC_adata.uns["rank_genes_groups"]  # Assuming marker genes are stored here
clus=4

deg_df = pd.DataFrame(index=[row[clus] for row in marker_genes['names']],
                      data={'scores':[ row[clus] for row in marker_genes['scores']],
                            'pvals':[ row[clus] for row in marker_genes['pvals']],
                            'pvals_adj':[ row[clus] for row in marker_genes['pvals_adj']],
                            'logfoldchanges':[ row[clus] for row in marker_genes['logfoldchanges']],
                            })
deg_df.to_csv(f'Cluster_{clus}_deg_df.csv')
fig_volcano, ax = plt.subplots(1, 1, figsize=(8, 4))
single_deg_volcano(deg_df,min_p=-1e-30,maxlfc=4,ax=ax,title=f'Cluster {clus}')


marker_genes = DGC_adata.uns["rank_genes_groups"]  # Assuming marker genes are stored here
clus=0

deg_df = pd.DataFrame(index=[row[clus] for row in marker_genes['names']],
                      data={'scores':[ row[clus] for row in marker_genes['scores']],
                            'pvals':[ row[clus] for row in marker_genes['pvals']],
                            'pvals_adj':[ row[clus] for row in marker_genes['pvals_adj']],
                            'logfoldchanges':[ row[clus] for row in marker_genes['logfoldchanges']],
                            })
deg_df.to_csv(f'Cluster_{clus}_deg_df.csv')
fig_volcano, ax = plt.subplots(1, 1, figsize=(8, 4))
single_deg_volcano(deg_df,min_p=-1e-30,maxlfc=4,ax=ax,title=f'Cluster {clus}')

In [ ]:
def deg_set_up(adata,keys_of_interest):
    combo_key = '_'.join(keys_of_interest)
    adata.obs[combo_key] = adata.obs[keys_of_interest[0]]
    for i,v in enumerate(keys_of_interest[1:]):
        # to_add = adata.obs[v]
        adata.obs[combo_key] = adata.obs[combo_key].str.cat(adata.obs[v].astype(str),sep='_')
    return adata,combo_key


def deg_analysis(adata_temp,new_file_path,analysis_pairs,group_key,adata_name='volcano',log2fc_extrema=[-15,15],label_cutoff=1,to_plot=True):
    '''
    Takes in anndata RNA object and a stratifying key and returns
    DEGs based on given grouppings. Also builds volcano plots.    '''
    adata_temp.raw = adata_temp
    '''Perform statistics and write to DF'''
    # print(os.getcwd())
    deg_df_dict={}
    for key,pair in analysis_pairs.items():
        sc.tl.rank_genes_groups(adata_temp,use_raw=True,groupby=group_key,groups=pair,reference=pair[1],key_added=key,method='wilcoxon',tie_correct=True)
        deg_df = sc.get.rank_genes_groups_df(adata_temp, group=pair[0],key=key,pval_cutoff=1,log2fc_min=log2fc_extrema[0], log2fc_max=log2fc_extrema[1])
        deg_df = deg_df.set_index('names')
        deg_df_dict[key] = deg_df
        new_file_name = os.path.join(new_file_path,adata_name+'_'+key+'.csv')
        print(new_file_name)
        deg_df.to_csv(new_file_name)
    return adata_temp, deg_df_dict

def deg_volc_plot(deg_df_dict,gene_list=[],label_cutoff=0.1,suptitle='',lock_y_max=True,special_genes=[],use_adj=False):
    ''' Plot Volcanoes '''
    fig_volcano,ax=plt.subplots(1,len(deg_df_dict),figsize=(len(deg_df_dict)*1.5,2))
    if len(deg_df_dict) == 1:  ax=[ax]
    for key,df in deg_df_dict.items():
        key_ind = list(deg_df_dict.keys()).index(key)
        ax[key_ind].set_title(key.replace('_','\n'))
        genes= df.index
        lfc = df['logfoldchanges']
        if use_adj: pval = df['pvals_adj']
        else: pval = df['pvals']

        neg_log10_pval = -np.log10(pval)
        for i,g in enumerate(genes):
            if str(g) in gene_list:
                if not np.isnan(lfc[i]*neg_log10_pval[i]):
                    ax[key_ind].scatter(lfc[i],neg_log10_pval[i],c='k',s=3)
                    if pval[i]<label_cutoff:
                        ax[key_ind].text(lfc[i],neg_log10_pval[i],genes[i],rotation=45,fontsize=4)

                if g in special_genes:
                    if not np.isnan(lfc[i]*neg_log10_pval[i]):
                        ax[key_ind].scatter(lfc[i],neg_log10_pval[i],c='r',s=3)
                        ax[key_ind].text(lfc[i],neg_log10_pval[i],genes[i],rotation=45,fontsize=4,color='r')
    x_lim=[]
    y_lim=[]
    for a in ax:
        x_lim.append(a.get_xlim())
        y_lim.append(a.get_ylim())

    x_etr = np.max(np.abs(x_lim))
    y_etr = np.max(np.abs(y_lim))

    for a in ax:
        a.set_xlim(-x_etr*1.1,x_etr*1.1)
        if lock_y_max: a.set_ylim(0,y_etr*1.1)
        else: a.set_ylim(0,a.get_ylim()[1]*1.1)
        a.set_xlabel('Log2 Fold Change')
        if use_adj: a.set_ylabel('-log10(adj-pvalue)')
        else: a.set_ylabel('-log10(raw-pvalue)')
        a.axhline(-np.log10(0.05),c='k',linestyle=":",linewidth=.6,label='p=0.05')

    fig_volcano.suptitle(suptitle)
    plt.tight_layout()
    return fig_volcano,ax

def deg_heatmap(deg_dict,gene_list=[],ax=None,lfcmax = 2.5,ax_cbar=None,suptitle=''):

    try: ax.grid(visible=False)
    except:
        fig,axs = plt.subplots(1,2,figsize=(1+.5*len(deg_dict.keys()),len(gene_list)/14),width_ratios=[9,.5])

        ax=axs[0]
        ax_cbar = axs[1]
        ax.grid(visible=False)
    if len(gene_list)==0:gene_list = list()

    heat_map_null=np.zeros([len(gene_list),len(deg_dict)])
    heat_map_lfc = heat_map_null.copy()*np.nan
    heat_map_p = heat_map_null.copy()*np.nan
    for key,deg_df in deg_dict.items():
        key_id = list(deg_dict.keys()).index(key)
        for gene in gene_list:
            i_gene = gene_list.index(gene)
            try:
                heat_map_lfc[i_gene,key_id] = deg_df.loc[gene,'logfoldchanges']
            except:
                None
    cbar = ax.pcolorfast(heat_map_lfc,vmin=-lfcmax,vmax=lfcmax,cmap='bwr')
    ax.set_yticks(np.arange(len(gene_list))+0.5, labels=gene_list,rotation=0)
    ax.tick_params(length=0)

    fmt_xlabels = list(deg_dict.keys())
    fmt_xlabels = [xl.replace('_','\n') for xl in fmt_xlabels]
    ax.set_xticks(np.arange(len(deg_dict.keys()))+0.5, labels=fmt_xlabels,rotation=0)
    ax.grid(visible=False)


    for key,deg_df in deg_dict.items():
        key_id = list(deg_dict.keys()).index(key)
        for gene in gene_list:
            i_gene = gene_list.index(gene)
            try:
                p = deg_df.loc[gene,'pvals']
                if p<.1:
                    p_str = str(p)[:5]
                    l2fc = str(2**deg_df.loc[gene,'logfoldchanges'])[:3]
                    tag = f"{l2fc}X; p={p_str}"
                    tag = pval_to_star(p)
                    ax.text(key_id+.5,i_gene+.5,tag,ha='center',va='top',rotation=0)
            except: None

    ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)

    try:
        plt.colorbar(cbar,cax=ax_cbar)
        ax_cbar.set_ylabel('log2(Fold Change)')
    except: plt.colorbar(cbar)
    plt.suptitle(suptitle)
    plt.tight_layout()
    return plt.gcf(), ax, cbar

def pval_to_star(p,specifics={(.05,.1):'\'',(.05,1):'ns'},max_star=5):
    for range,tag in specifics.items():
        if p<=np.max(range) and p>np.min(range): return tag
    if p<=0.5 and p>0.01: return '*'
    nlogp=-np.log10(p)
    return ('*'*int(nlogp))[:max_star]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def single_deg_volcano(
    deg_df,
    label_cutoff=0.1,
    title=None,
    comp_string=None,
    lock_y_max=True,
    special_genes=[],
    use_adj=False,
    marker_size = .5,
    min_p = 1e-30,
    maxlfc =10,
    ax=None):
    '''
    Plot a single volcano plot for a DE comparison.

    Parameters:
        deg_df (DataFrame): DataFrame of DE results with gene names as index and columns including
                            'logfoldchanges', 'pvals', and optionally 'pvals_adj'.
        gene_list (list, optional): List of genes to be highlighted in the plot (default is an empty list).
        label_cutoff (float, optional): Minimum -log10(p-value) threshold for labeling genes (default is 0.1).
        title (str, optional): Title for the plot (default is an empty string).
        lock_y_max (bool, optional): If True, locks the y-axis max across subplots; adjust to data range otherwise.
        special_genes (list, optional): Genes to be specially highlighted in red (default is an empty list).
        use_adj (bool, optional): If True, use adjusted p-values; otherwise, use raw p-values (default is False).

    Returns:
        fig_volcano (Figure): Matplotlib figure object for the volcano plot.
        ax (Axes): Matplotlib axes object for the volcano plot.
    '''

    # Initialize the figure and axis for the volcano plot
    if ax is None:
        fig_volcano, ax = plt.subplots(1, 1, figsize=(1.75, 1.75))
    else:
        fig_volcano = ax.figure

    # Set the title for the plot
    if title is not None:
        ax.set_title(title.replace('_', '\n'))

    # Extract necessary data
    genes = deg_df.index
    lfc = deg_df['logfoldchanges']
    pval = deg_df['pvals_adj'] if use_adj else deg_df['pvals']
    pval[pval<min_p]=min_p
    lfc[lfc<-maxlfc]=-maxlfc
    lfc[lfc>maxlfc]=maxlfc
    neg_log10_pval = -np.log10(pval)

    # Plot each gene, marking those in gene_list and special_genes
    ax.scatter(lfc, neg_log10_pval, c='gray', s=marker_size)

    is_sig = neg_log10_pval>4
    high_lfc = abs(lfc)>=1

    selected = np.logical_and(is_sig,high_lfc)

    lfc_sig = lfc[selected]
    neg_log10_pval_sig = neg_log10_pval[selected]
    genes_sig = genes[selected]

    for i, g in enumerate(genes_sig):
        if not np.isnan(lfc_sig[i] * neg_log10_pval_sig[i]):
            ax.scatter(lfc_sig[i], neg_log10_pval_sig[i], c='r', s=marker_size)
            ax.text(lfc_sig[i], neg_log10_pval_sig[i], genes_sig[i], rotation=45, fontsize=4, color='r')

    # Set axis limits and labels
    x_lim = ax.get_xlim()
    y_lim = ax.get_ylim()
    x_etr = np.max(np.abs(x_lim))
    y_etr = np.max(np.abs(y_lim))

    ax.set_xlim(-x_etr * 1.1, x_etr * 1.1)
    if lock_y_max:
        ax.set_ylim(0, y_etr * 1.1)
    else:
        ax.set_ylim(0, ax.get_ylim()[1] * 1.1)

    if comp_string is not None:
        xlabel = f'{comp_string}\nLog2(fold change)'
    else:
        xlabel = 'Log2(fold change)'
    ax.set_xlabel(xlabel)
    ax.set_ylabel('-log10(adj-pvalue)' if use_adj else '-log10(raw-pvalue)')
    ax.axhline(-np.log10(0.05), c='k', linestyle=":", linewidth=0.6, label='p=0.05')
    ax.axvline(0, c='k', linestyle="-", linewidth=0.6, label='p=0.05')

    plt.tight_layout()

    return fig_volcano, ax


